## The Traveling Salesman Problem

with integer programming and Gurobi

Source: http://examples.gurobi.com/traveling-salesman-problem/

经典的TSP组合优化问题 In this example we'll solve the Traveling Salesman Problem.

We'll construct a mathematical model of the problem, implement this model in Gurobi's Python interface, and compute and visualize an optimal solution.

Although your own business may not involve traveling salesmen, the same basic techniques used in this example can be used for many other applications like vehicle rounting, circuit design and DNA sequencing.

<img height=400 width=600 src=http://examples.gurobi.com/traveling-salesman-problem/screenshot.png>


The Traveling Salesman Problem (TSP) is a classic problem in combinatorial optimization. It was first formulated as an integer program by Dantzig, Fulkerson and Johnson in 1954.

一个销售员在美国旅行，从纽约出发。寻找最短路线。

In this example, we consider a salesman traveling in the US. The salesman starts in New York and has to visit a set of cities on a business trip before returning home. The problem then consists of finding the shortest tour which visits every city on the itinerary.

使用 lazy constraints ？

We will formulate this problem as an integer program and implement it in Gurobi. The implementation will also demonstrate the use of lazy constraints in Gurobi.

## Problem Description



## Implementation

In [1]:
import math
import random
from gurobipy import *


# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
  if where == GRB.callback.MIPSOL:
    selected = []
    # make a list of edges selected in the solution
    for i in range(n):
      sol = model.cbGetSolution([model._vars[i,j] for j in range(n)])
      selected += [(i,j) for j in range(n) if sol[j] > 0.5]
    # find the shortest cycle in the selected edge list
    tour = subtour(selected)
    if len(tour) < n:
      # add a subtour elimination constraint
      expr = 0
      for i in range(len(tour)):
        for j in range(i+1, len(tour)):
          expr += model._vars[tour[i], tour[j]]
      model.cbLazy(expr <= len(tour)-1)


# Euclidean distance between two points

def distance(points, i, j):
  dx = points[i][0] - points[j][0]
  dy = points[i][1] - points[j][1]
  return math.sqrt(dx*dx + dy*dy)


# Given a list of edges, finds the shortest subtour

def subtour(edges):
  visited = [False]*n
  cycles = []
  lengths = []
  selected = [[] for i in range(n)]
  for x,y in edges:
    selected[x].append(y)
  while True:
    current = visited.index(False)
    thiscycle = [current]
    while True:
      visited[current] = True
      neighbors = [x for x in selected[current] if not visited[x]]
      if len(neighbors) == 0:
        break
      current = neighbors[0]
      thiscycle.append(current)
    cycles.append(thiscycle)
    lengths.append(len(thiscycle))
    if sum(lengths) == n:
      break
  return cycles[lengths.index(min(lengths))]

n = 50

# Create n random points

random.seed(1)
points = []
for i in range(n):
  points.append((random.randint(0,100),random.randint(0,100)))

m = Model()


# Create variables

vars = {}
for i in range(n):
   for j in range(i+1):
     vars[i,j] = m.addVar(obj=distance(points, i, j), vtype=GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))
     vars[j,i] = vars[i,j]
   m.update()


# Add degree-2 constraint, and forbid loops

for i in range(n):
  m.addConstr(quicksum(vars[i,j] for j in range(n)) == 2)
  vars[i,i].ub = 0

m.update()


# Optimize model

m._vars = vars
m.params.LazyConstraints = 1
m.optimize(subtourelim)

solution = m.getAttr('x', vars)
selected = [(i,j) for i in range(n) for j in range(n) if solution[i,j] > 0.5]
assert len(subtour(selected)) == n

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 50 rows, 1275 columns and 2500 nonzeros
Variable types: 0 continuous, 1275 integer (1275 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 2681.8
Presolve removed 0 rows and 50 columns
Presolve time: 0.00s
Presolved: 50 rows, 1225 columns, 2450 nonzeros
Variable types: 0 continuous, 1225 integer (1225 binary)

Root relaxation: objective 6.046285e+02, 83 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  604.62853    0    6 2681.79584  604.62853  77.5%     -    0s
H    0     0                     692.2886515  604.62853  12.7%     -    0s
     0     0  614.09342    0   12  692.28865  614.0934

## Optimal Result


In [2]:
selected

[(0, 8),
 (0, 36),
 (1, 29),
 (1, 41),
 (2, 32),
 (2, 34),
 (3, 22),
 (3, 46),
 (4, 13),
 (4, 17),
 (5, 11),
 (5, 29),
 (6, 9),
 (6, 19),
 (7, 27),
 (7, 33),
 (8, 0),
 (8, 39),
 (9, 6),
 (9, 45),
 (10, 30),
 (10, 36),
 (11, 5),
 (11, 23),
 (12, 16),
 (12, 21),
 (13, 4),
 (13, 28),
 (14, 27),
 (14, 32),
 (15, 17),
 (15, 38),
 (16, 12),
 (16, 44),
 (17, 4),
 (17, 15),
 (18, 24),
 (18, 48),
 (19, 6),
 (19, 35),
 (20, 26),
 (20, 47),
 (21, 12),
 (21, 28),
 (22, 3),
 (22, 31),
 (23, 11),
 (23, 37),
 (24, 18),
 (24, 31),
 (25, 30),
 (25, 44),
 (26, 20),
 (26, 46),
 (27, 7),
 (27, 14),
 (28, 13),
 (28, 21),
 (29, 1),
 (29, 5),
 (30, 10),
 (30, 25),
 (31, 22),
 (31, 24),
 (32, 2),
 (32, 14),
 (33, 7),
 (33, 42),
 (34, 2),
 (34, 49),
 (35, 19),
 (35, 38),
 (36, 0),
 (36, 10),
 (37, 23),
 (37, 47),
 (38, 15),
 (38, 35),
 (39, 8),
 (39, 42),
 (40, 43),
 (40, 48),
 (41, 1),
 (41, 45),
 (42, 33),
 (42, 39),
 (43, 40),
 (43, 49),
 (44, 16),
 (44, 25),
 (45, 9),
 (45, 41),
 (46, 3),
 (46, 26),
 (47, 